In [128]:
import flask, os, io, h5py
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
from skimage.restoration import (denoise_wavelet, estimate_sigma)

In [81]:
metadata = pd.read_csv('data_temp.csv')
metadata.head()

,Station,Source,Net,Station_Latitude,Station_Longitude,Distance,Azimuth,Elevation,Station Name,Start_Time,Station_Elevation,Station_Depth,Earthquake_Latitude,Earthquake_Longitude,Earthquake_Depth,Earthquake_Station_Distance,Back_Azimuth
0,COCO,IRISDMC,II,-12.1901,96.8349,8.622234,-143.996826,1.0,"West Island, Cocos (Keeling) Islands",2022-05-16T18:50:28.019538Z,1.0,44.799999,-5.245,102.007698,23.0,954.800049,36.965694
1,KAPI,IRISDMC,II,-5.0142,119.7517,17.673860,90.057363,300.0,"Kappang, Sulawesi, Indonesia",2022-05-16T18:52:29.019538Z,300.0,61.000000,-5.245,102.007698,23.0,1965.260376,268.468445
2,MBWA,IRISDMC,IU,-21.1590,119.7313,23.422910,134.423008,190.0,"Marble Bar, Western Australia",2022-05-16T18:53:32.019538Z,93.0,97.000000,-5.245,102.007698,23.0,2597.827881,310.139557
3,CHTO,IRISDMC,IU,18.8141,98.9443,24.247711,-7.075346,420.0,"Chiang Mai, Thailand",2022-05-16T18:53:40.019538Z,320.0,100.000000,-5.245,102.007698,23.0,2679.332520,172.510071
4,DAV,IRISDMC,IU,7.0697,125.5791,26.551312,62.599605,151.0,"Davao, Philippines",2022-05-16T18:54:01.019539Z,150.0,1.000000,-5.245,102.007698,23.0,2948.098145,243.135620


In [ ]:
data_station = {}
for idx_ in range(len(metadata)):
    each_station = {}
    station_ = metadata.loc[idx_, 'Station']
    station_location_ = metadata.loc[idx_, 'Station Name']
    lat_ = metadata.loc[idx_, 'Station_Latitude']
    long_ = metadata.loc[idx_, 'Station_Longitude']
    elevation_ = metadata.loc[idx_, 'Elevation']
    
    each_station.update({
        'Station':station_,
        'Station Location':station_location_,
        'Latitude':lat_,
        'Longitude':long_,
        'Elevation':elevation_
    })
    data_station.update({f'Station {idx_+1}':each_station})

In [ ]:
data_station

In [2]:
#create an instance of Flask
app = flask.Flask('Earthquake Model Deployment')
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = True

@app.route('/')
def home():
    return "Halo dunia tipu tipu!"

@app.route('/station/')
def channel_station():
    metadata = pd.read_csv('data_temp.csv')
    data_station = {}
    for idx_ in range(len(metadata)):
        each_station = {}
        station_ = metadata.loc[idx_, 'Station']
        station_location_ = metadata.loc[idx_, 'Station Name']
        lat_ = metadata.loc[idx_, 'Station_Latitude']
        long_ = metadata.loc[idx_, 'Station_Longitude']
        elevation_ = metadata.loc[idx_, 'Elevation']

        each_station.update({
            'Station':station_,
            'Station Location':station_location_,
            'Latitude':lat_,
            'Longitude':long_,
            'Elevation':elevation_
        })
        data_station.update({f'Station {idx_+1}':each_station})
    return flask.jsonify(data_station)


app.run(debug=False, port=8080)

 * Serving Flask app 'Earthquake Model Deployment' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8080 (Press CTRL+C to quit)
127.0.0.1 - - [11/Jun/2022 21:24:40] "GET /station/ HTTP/1.1" 200 -


In [60]:
# === LOAD SAVEDMODEL ===

class F1_Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.f1 = self.add_weight(name='f1', initializer='zeros')
        self.precision_fn = tf.keras.metrics.Precision(thresholds=0.5)
        self.recall_fn = tf.keras.metrics.Recall(thresholds=0.5)
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        p = self.precision_fn(y_true, y_pred)
        r = self.recall_fn(y_true, y_pred)
        self.f1.assign(2 *((p*r)/(p + r + 1e-6)))
        
    def result(self):
        return self.f1

    def reset_state(self):
        self.precision_fn.reset_states()
        self.recall_fn.reset_states()
        self.f1.assign(0)

PATH = 'C://Users/Zhafran/Documents/Data Science/BANGKIT - Capstone - Earthquake/Model Checkpoint/v1.1 (P-Wave Final)'
load_model = tf.keras.models.load_model(PATH, custom_objects={'F1_Score':F1_Score,
                                                              'loss':tfa.losses.SigmoidFocalCrossEntropy()})

In [164]:
def DenoiseWavelet(data, type='BayesShrink'):
    def BayesShrink():
        im_bayes = denoise_wavelet(np.array(data), convert2ycbcr=True, multichannel=True,
                                  method='BayesShrink', mode='soft', 
                                  rescale_sigma=True, wavelet_levels=4)
        return im_bayes
    
    def VisuShrink():
        sigma_est = estimate_sigma(np.array(data), multichannel=True, average_sigmas=True)
        im_visu = denoise_wavelet(np.array(img), convert2ycbcr=True, multichannel=True,
                                  method='VisuShrink', mode='soft', wavelet_levels=4,
                                  sigma=sigma_est, rescale_sigma=True)
        
        return im_visu
    
    if type=='BayesShrink':
        return BayesShrink()
    elif type=='VisuShrink':
        return VisuShrink()

def create_feature(data_feature):
    rs = []
#     data_l = get_waveform(data_feature)
    data_denoise = DenoiseWavelet(data_feature, type='BayesShrink')
#     print(f'data denoise shape: {data_denoise.shape}')
    dt = tf.keras.utils.timeseries_dataset_from_array(data=data_denoise, targets=None,
                                                          sequence_length=100, sequence_stride=20,
                                                          shuffle=False)
    for i in dt:
        rs.append(i)
        
    rs = tf.stack(rs)
    feature_rs = tf.data.Dataset.from_tensor_slices(rs)
    feature_rs = feature_rs.cache().batch(56).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return feature_rs

In [74]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, x_set, batch_size):
        self.x = x_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x


In [75]:
def get_predict_p(arr, threshold=.5, types='Highest'):
    arr = arr.reshape(-1)
    
    if types=='Highest':
        if np.max(arr)>=threshold:
            return np.argmax(arr)*20+275
        else:
            return -1
    elif types=='Early':
        for i, p_prob in enumerate(arr):
            if p_prob>=threshold:
                return i*20+275
        else:
            return -1
    elif types=='Late':
        i_thres = None
        for i, p_prob in enumerate(arr):
            if p_prob>=threshold:
                i_thres = i
        if i_thres!=None:
            return i_thres*20+275
        else:
            return -1
      

In [119]:
mean_ = np.mean(X[1, :, 0])
X[1, :, 0]-mean_

array([216.92195, 202.59862, 376.27274, ..., 364.9865 , 547.75824,
       430.6273 ], dtype=float32)

In [154]:
X = np.load('C://Users/Zhafran/Documents/Data Science/BANGKIT - Capstone - Earthquake/data_wavelength (1).npz')['arr_0']
X.shape

(3, 180000, 3)

In [165]:
%%time
# X = np.load(io.BytesIO(data))['arr_0']
index_ = 2
print(X[index_].shape)
# data_pred = create_feature(X[index_])

for i in range(180000//1200):
    start_, end_ = 0, 1200
    data_pred = create_feature(X[index_, start_:end_, :])
    y_pred = load_model.predict(data_pred)
    
    high_ = get_predict_p(y_pred, types='Highest')
    early_ = get_predict_p(y_pred, types='Early')
    late_ = get_predict_p(y_pred, types='Late')
    
    if (high_ and early_ and late_) == -1:
        start_+=1200
        end_+=1200
        continue
    
    print(f'Epochs: {i+1}')
    print(f'High Probability : {high_} Second')
    print(f'Early Probability: {early_} Second')
    print(f'Late Probability : {late_} Second\n')
    
    start_+=1200
    end_+=1200

(180000, 3)


C:\Users\Zhafran\AppData\Local\Temp/ipykernel_8920/1413613744.py:3: FutureWarning: `multichannel` is a deprecated argument name for `denoise_wavelet`. It will be removed in version 1.0. Please use `channel_axis` instead.
  im_bayes = denoise_wavelet(np.array(data), convert2ycbcr=True, multichannel=True,


Wall time: 12.1 s


In [31]:
from google.cloud import storage
from google.oauth2 import service_account

BASE_PATH = 'C://Users/Zhafran/Documents/Data Science/BANGKIT - Capstone - Earthquake/Keys/Cloud Storage'
PATH = BASE_PATH+'/bangkit-351503-314e5d9c4252.json'
credentials = service_account.Credentials.from_service_account_file(PATH)

client = storage.Client(credentials=credentials)

# Retrieve an existing bucket
# https://console.cloud.google.com/storage/browser/[bucket-id]/
bucket = client.get_bucket('data-gempa')

# Then do other things...
blob = bucket.get_blob('data_wavelength.npz')
data = blob.download_as_string()
# df = pd.read_csv(io.BytesIO(data))
# df.head()

In [33]:
type(data)

bytes

In [58]:
np.load(io.BytesIO(data))['arr_0'][0]

array([[  80.28691  ,   15.379599 ,  -42.924072 ],
       [ 100.01992  ,   -3.0439146,  -66.021286 ],
       [  97.994774 ,  -22.929815 ,  -66.9469   ],
       ...,
       [-143.53304  ,  101.31471  ,   31.710686 ],
       [ -71.961945 ,   64.44231  ,   25.03955  ],
       [  -3.2141392,   31.379045 ,    3.169337 ]], dtype=float32)

In [59]:
data_pred = create_feature(np.load(io.BytesIO(data))['arr_0'][0])
data_pred

NameError: name 'create_feature' is not defined

In [129]:
def get_waveform(trace_name):
    '''
    Kolom 1: East-West Channel
    Kolom 2: North-South Channel
    Kolom 3: Z (Vertical) Channel
    '''
    filename = "merge.hdf5"
    with h5py.File(filename, "r") as f:
        data = f.get('data/'+trace_name)
        data = np.array(data)
    return data

In [130]:
aaaa = pd.read_csv('merge.csv')
aaaa.head()

C:\Users\Zhafran\.conda\envs\ExVodka\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (7,11,13,14,15,18,19,20,21,22,24,25,26,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,network_code,receiver_code,receiver_type,receiver_latitude,receiver_longitude,receiver_elevation_m,p_arrival_sample,p_status,p_weight,p_travel_sec,...,source_magnitude_author,source_mechanism_strike_dip_rake,source_distance_deg,source_distance_km,back_azimuth_deg,snr_db,coda_end_sample,trace_start_time,trace_category,trace_name
0,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-21 05:55:00,noise,109C.TA_201510210555_NO
1,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-11-06 14:50:00,noise,109C.TA_201511061450_NO
2,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-11-07 02:20:00,noise,109C.TA_201511070220_NO
3,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-11-14 05:15:00,noise,109C.TA_201511140515_NO
4,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-12-25 18:50:00,noise,109C.TA_201512251850_NO


In [144]:
bbbb = get_waveform(aaaa.loc[994927, 'trace_name'])
np.max(bbbb)

352.21976